# 2

The purpose of this homework is to get acquainted with the solution of linear systems using elementary methods. As a model problem, we will work with polynomial interpolation. We will also use this homework as an opportunity to examine and familiarize ourselves with the Numpy library for Python, which makes it extremely easy to work with arrays in general, and vectors and matrices in particular.

- - -

<div class="alert alert-info">

### NumPy and Linear Algebra in Python
</div>

There are several excellent sources of documentation for Numpy.

<div class="alert alert-success">
    
**Task 1:** Familiarize yourself with Numpy by reading the [quickstart tutorial](https://docs.scipy.org/doc/numpy/user/quickstart.html):
- In particular, since we will be working with matrices (two-dimensional arrays), make sure you understand indexing and slicing.
- Also, be aware of the semantics of assignments (view vs. copy), which is sometimes tricky for beginners. More [here](https://www.jessicayung.com/numpy-views-vs-copies-avoiding-costly-mistakes/) and [here](https://scipy-cookbook.readthedocs.io/items/ViewsVsCopies.html).
</div>

Let's get started by importing the relevant packages and doing some basic setup.

In [ ]:
import numpy as np              # basic arrays, vectors, matrices
import numpy.linalg as la       # linear algebra functions
import matplotlib.pyplot as plt # plotting

%matplotlib inline

- - -

<div class="alert alert-info">

### Polynomial Interpolation
</div>

*Interpolation* refers to the process of extrapolating from discrete measurements to a continuous function. A model problem is for example to compute the continuous trajectory $x(t)$ of an object over time from discrete snapshots $(t_i,x_i)$. *Polynomial interpolation* specifically refers to describing the continuous function as a polynomial.

Formally, the (one-dimensional) polynomial interpolation problem is described as follows: given a sequence of $n+1$ $x$-coordinates $(x_0,\dots,x_n)$ with $x_i \neq x_j$ if $i\neq j$ and corresponding values $y_i \in \mathbb{R}$, determine a polynomial $p(x)$ of degree $n$ such that

$$
p(x_i)\ =\ a_n x_i^n + a_{n-1} x_i^{n-1} + \cdots + a_1 x_i + a_0\ \stackrel{!}{=}\ y_i,\ \ i=0,\dots,n.
$$

This problem can be turned into a linear system, as follows:

Let $v_i$ the vector of powers of $x_i$, i.e. 

$$v_i = (1, x_i, x_i^2, \dots, x_i^n)^T,$$ 

and let $a = (a_0, a_1, \dots, a_n)^T$ the vector of coefficients of $p$. Then, the interpolation condition $p(x_i) = y_i$ can be written as the scalar product of $v_i$ and $a$, since

$$
p(x_i)\ =\ a_n x_i^n + \cdots + a_1 x_i + 1 \cdot a_0 = \ a^T v_i\  \stackrel{!}{=} y_i
$$

Repeating this for $i=0,\dots,n$, we obtain a linear system for the coefficients:

$$
\begin{pmatrix}
1 & x_0 & x_0^2 & \cdots & x_0^n \\
1 & x_1 & x_1^2 & \cdots & x_1^n \\
1 & x_2 & x_2^2 & \cdots & x_2^n \\
\vdots & \vdots &\vdots &\vdots & \\
1 & x_n & x_n^2 & \cdots & x_n^n \\
\end{pmatrix}
\begin{pmatrix} a_0 \\ a_1 \\ a_2 \\ \vdots \\ a_n\end{pmatrix} =
\begin{pmatrix} y_0 \\ y_1 \\ y_2 \\ \vdots \\ y_n\end{pmatrix}
\qquad
$$

or briefly using matrix notation

$$V a = y$$

The $(n+1)\times (n+1)$ system matrix $V$ with $V_{ij} = x_i^j$ is called a *Vandermonde matrix*. A solution of the corresponding system gives the coefficients $a$ of the polynomial $p$.

Let's consider a specific problem instance: first, define NumPy arrays containing the data $x_i$ and $y_i$:

In [ ]:
n = 4
y = np.array( [3., 5, 2, 3, 1] )
x = np.array( [0., 1, 2, 3, 4] )

plt.plot( x, y, 'bo' );

Graphically, our goal is to construct a polynomial of degree $4$ (since there are five points) that exactly passes through these points. 

(You have encountered this problem previously when constructing a line (= polynomial of degree one) given two points, or a parabola (= polynomial of degree two) that passes through three given points.

We begin by constructing the Vandermonde system matrix for the $x_i$. Naively, we would write

In [ ]:
V = np.empty( (n+1, n+1) )

# directly fill in matrix entries
for i in range(n+1):
    for j in range(n+1):
        V[i,j] = x[i]**j
        
V

However, we can write this a little more compactly using a list comprehension:

In [ ]:
# create a 2D array whose rows are the powers of x,
# then transpose so the columns are the powers of x
V = np.array( [x**p for p in range(n+1)] )
V

Even shorter: NumPy directly offers the [`vander`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.vander.html) function for constructing $V$. By default, `vander` constructs the matrix such that the powers decrease along the rows of $V$; we thus need to pass `increasing=True` to match the order of $a$ we defined above.

In [ ]:
V = np.vander( x, increasing=True )
V

Now we can solve the system $Va = y$ for $a$ to obtain the coefficients of the polynomial $p$.

- - -

<div class="alert alert-info">
    
### Gauss Elimination
</div>

As discussed in the lecture, the Gauss elimination algorithm can be used to solve the system above.

<div class="alert alert-success">
    
**Task 2:** Implement the Gauss algorithm (without pivoting) in two steps through the below functions:
- `bwd_subs` should perform backward substitution to solve an upper tridiagonal system
- `gauss_solve` should reduce the extended matrix $(A,b)$ to upper tridiagonal form via Gauss elimination, and then use `bwd_subs` to solve the system
- It is always sensible to think about ways to validate your implementations, e.g. check that `bwd_subs` does the right thing.
- Note that while indices of matrix rows etc. start at $1$ in standard mathematical notation (and hence in the slides), NumPy indices begin at $0$.
</div>

In [ ]:
"""
U (list of lists): Upper triangular matrix.
y (list): Right-hand side vector.
https://wichit2s.gitlab.io/numer/ch02/index.html#back-substitution-in-python ; Title: 2.12. Back Substitution in Python; 12.11.2023.
"""
def bwd_subs( U, y ):
    n = len(U)
# To ensure that the matrix is constructed like seen in the output below this cell.
    sol = np.empty_like(y, dtype=float)
# range(start, until, steps) iterates over.
    for i in range(n-1, -1, -1):
        sol[i] = (y[i] - np.dot(U[i,i+1:n], sol[i+1:n]))/U[i,i]# sol[i] = (y[i] - np.dot(U[i, i+1:], sol[i+1:, i])) / U[i, i]
    return sol



'''
Reduce matrix (A,b) to upper tridiagonal form via Gauss elimination, then use bwd_subs to solve.
Idea: make the diagonal element (U[i][i]) equal to 1 by dividing the entire row by that element.
Inspiration https://stackoverflow.com/questions/58615531/gaussian-elimination-no-pivoting
'''
def gauss_solve( V, y ):
# We can: exchange two rows/ columns, add /substract multiple of one row to/from another row.
    assert len(V) == len(y), "Dimensions of U and y must be the same"
    n = len(y)
    y = np.array(y)

    for row in range(n):
        for i in range(row + 1, n):
            fac = V[i, row] / V[row, row]  # Use integer division here
            V[i, row:] = V[i, row:]- (fac * V[row, row:])
            y[i] = y[i] - (fac * y[row])
    ret = bwd_subs(V, y) # np.hstack() bringt hier nichts, weil die Ausgabe 1D ist.
    return ret

# Example from the lecture. Page 42.
ü = np.array([[4, 2, 2], # (3,3) shape
     [0, -16, -4],
     [0, 0, 2]])

ä = np.array([10, -24, 4]) # (3,) shape

z = np.array([[4, 2, 2],
     [8, -12, 0],
     [-4, 14, 4]])

o = np.array([10, -4, 18])

v = np.array([1, 1, 2, 3, 5, 8, 13, 21]) # (8,) shape

print ("Output:", bwd_subs(ü, ä), '\n')
# On the left the known solution and on the right the output. Gives Boolean back.
print ("The solution is", [1,1,2] == bwd_subs(ü, ä), '\n')

#Tests für den Gaußalgo
print(gauss_solve(ü, ä), '\n')
print(gauss_solve(z, o), '\n')

print(np.shape(z), '\n')

print(np.shape(gauss_solve(z, o)), '\n')

print(np.shape(ä))

print(np.shape(v))


Let's apply this to our problem and validate the solution graphically. NumPy conveniently provides the [`polyval`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.polyval.html) function to evaluate the found polynomial. 

A potential showstopper is that `polyval` requires as input the coefficients in descending order (i.e. `a[0]` as the coefficient of $x^n$), while we compute them in ascending order (`a[n]` is the coefficient of $x^n$). This is however easily remedied using NumPy's [`flip`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.flip.html?highlight=flip#numpy.flip) function.

In [ ]:
a = gauss_solve( V, y )

# plot output
xx = np.linspace( x.min(), x.max(), 1000 )
yy = np.polyval( np.flip(a), xx )

plt.plot( xx, yy, 'r-', x, y, 'bo' );

Problem solved! (... if you completed task 2.)

- - -

<div class="alert alert-info">
    
### LU Decomposition
</div>

While Gauss elimination works well for a small number of points, for a large number of points it does not work well. Let's look at a larger problem instance:

In [ ]:
n = 50
x = np.linspace( -1, 1, n )
y = np.cos( 15*x**2 )

# set up system matrix
V = np.vander( x, increasing=True )
a = gauss_solve( V, y )

# plot output
xx = np.linspace( x.min(), x.max(), 100 )
yy = np.polyval( np.flip(a), xx )

plt.plot( xx, yy, 'r-', x, y, 'bo' );

This does not look entirely right. In order to quantify the error, we can compute the norm of the *residual* vector: 

$$r := y - Va $$

In [ ]:
la.norm(y- np.matmul(V, a))

and see that it is not actually small. 

The reason that the residual is so large lies in the ill-conditioned nature of the system matrix. NumPy gives us an easy way to compute the condition number of the system matrix, which indicates how much small errors are amplified when solving the system:

In [ ]:
la.cond( V )

That is a big number. While the Gauss algorithm works well for well-conditioned problems, it is not stable enough to address severely ill-conditioned problems; hence it does not work well here.

**Aside**: It is easy to see that the condition number increases rapidly with $n$:

In [ ]:
plt.plot( [np.log10(la.cond(np.vander(np.linspace(0,1,n)))) for n in range(1,20)] );

As the number of points increases, the columns of the Vandermonde matrix become more and more similar and thus less and less linear independent; for 10 points, the condition number is already $\approx 10^8$. In a sense, the system becomes more and more difficult to solve, leading to increasing problems in the application of numerical techniques. (This is quite typical behavior for many important numerical problems / techniques.)

To remedy this, let's apply the LU decomposition with partial pivoting to solve this system. The purpose of the pivoting is to increase the numerical stability of the algorithm.

Here is a slightly simpler algorithm for the LU decomposition than the one discussed in the lecture. It decomposes $PA = LU$ and does not detect singular $A$, but is easier to implement:

- - -
    
Input $A \in \mathbb{R}^{n \times n}$; Output $P,L,U$ such that $PA = LU$.

Let $U = I_n, L = I_n, P = I_n$ with $I_n$ being the identity matrix of dimension $n \times n$.

For $k = 1, \ldots, n$:
- find $k \leq i \leq n$ to maximize $|A_{ik}|$
- exchange rows $k$ and $i$ of $P$

Let $A' = P \cdot A$.

For $j = 1, \ldots, n$:
- For $i = 1, \ldots, j$: $U_{ij} = A'_{ij} - \sum\limits_{k=1}^{i-1}{U_{kj}L_{ik}}$
- For $i = j, \ldots, n$: $L_{ij} = \frac{1}{U_{jj}} \left(A'_{ij} - \sum\limits_{k=1}^{j-1}{U_{kj}L_{ik}}\right)$

- - -

<div class="alert alert-success">

**Task 3**: implement the routines `lu_factor`, `fwd_subs` and `lu_solve` below using the approach given above.

</div>

In [ ]:
def lu_factor(A):
    """Perform an LU decomposition PA = LU with partial pivoting of the square matrix A."""
    
    n = A.shape[0]
    AA = np.copy(A) # just a copy of the given matrix. we just use it to check down if PA=LU

    P = np.identity(n)
    U = np.identity(n)
    L = np.identity(n)
    
    for k in range(n):
        # Here we find row i to maximize |A_ik|.
        # it does that by taking the k-th row in the column that has the highest absolute value (with np.abs)
        # np.argmax then gets the index of that highest value, and we and k (+k on the right) to get the right index(given that we started at the k-th row)
        i_row = np.argmax(np.abs(A[k:, k])) + k
        
        if i_row != k: #It might happen that the current row in column k already has the max value, and is as such equal to i_row. In that case we do nothing.
            P[[k, i_row]] = P[[i_row, k]] # we exchange rows k and i of matrix P
            
    A_prime = P@A
    
    # the exact algorithm in the previous cell
    for j in range(n):
        for i in range(j+1):
            U[i, j] = A_prime[i, j] - sum(U[k, j] * L[i, k] for k in range(i))
            
        for i in range(j, n):
            L[i, j] = (1/(U[j, j])) * (A_prime[i, j] - sum(U[k, j] * L[i, k] for k in range(j)))

#     print(P@AA, np.all(P@AA== L@U), '\n') #print out PA and testing if it is in fact equal to LU
#     print(P@L@U) # printing PLU to see if it is equal to A
    
    return P, L, U

def fwd_subs( L, b ):
    """Solve the linear system Ly = b with lower triangular matrix L by forward substitution."""
    y = np.empty_like(b) 
    
    n = len(L) #we get the length of L
    
    for i in range(n): 
        y[i] = (b[i] - (L[i, :i])@(y[:i])) / L[i, i] # works like bwd_subs, but multiplies the matrices from their first columns up till, but not including the i-th column.
            
    return y

def lu_solve(P, L, U, b):
    """Solve the linear system Ax = b with upper/lower triangular matrices U, L and
       permutation matrix P."""
#     print(U)
#     print('\n and np.triu(U) is \n')
#     print(np.triu(U))
    assert np.all(U == np.triu(U)) # ensure U is upper triangular.  
    #...

    c = fwd_subs(L, (P@b)) # solve for c using forward substitution and the lower triangular matrix.
    x = bwd_subs(U, c) # solve for x using c and the upper triangular matrix.    

    return x

In [ ]:
# N = np.vander([1,2,3,4], increasing=True)
# M = np.vander([3,4,5,6], increasing=True)

# N = np.array([[1,0,0],[1,1,0],[1,1,1]])
# M = np.vander([1,1,1])
# print(np.triu(N))
# print(M)

# print(np.eye(len(N)))

# temp = np.copy(N[1, :])
# N[1, :] = M[1, :]
# M[1, :] = temp

# N[[1, 2]] = N[[2, 1]]


# M[[1, 2]] = M[[2, 1]]
# print(M, '\n')

# for k in range(len(M)):
#     i_row = np.argmax(np.abs(M[k:, k])) + k
#     print(k, i_row)
# #     if i_row != k:
#     print('replace', M[k, :], 'by', M[i_row, :])
#     M[[k, i_row]] = M[[i_row, k]]
    
    
#     temp = np.copy(M[k, :])
#     M[k, :] = M[i_row, :]
#     M[i_row, :] = temp
# max = np.argmax(np.abs(M[2:, 2]))

# print(N, '\n')
# print(M, '\n')
# print(np.argmax(M[3:, 3]))
# for j in range (4):
#     for i in range(j):
#         print(i, 'i1')
#     for i in range(j, 4):
#         print(i, 'i2')


# print(N.shape, '\n') 
# M = np.identity(N.shape[0])
# print(M, '\n')
# Y = np.linalg.inv(N)
# print(Y, '\n')
# print(N@Y)

# N = np.array([[4, 2, 4],
#              [8, -12, 0],
#              [-4, 14, 4]])

# print(N, '\n')
# print(np.triu(M))

# print(lu_factor(N))

Let's apply this to our problem:

In [ ]:
n = 50
x = np.linspace( -1, 1, n )
y = np.cos( 15*x**2 )

V = np.vander( x, increasing=True )

P,L,U = lu_factor(V)
a = lu_solve(P,L,U,y)

print( "residual = ", la.norm( y- np.matmul(V, a)) )

# plot output
xx = np.linspace( x.min(), x.max(), 200 )
yy = np.polyval( np.flip(a), xx )

plt.plot( xx, yy, 'r-', x, y, 'bo' );

- - - 

<div class="alert alert-info">

### Microbenchmarks
</div>

Beyond numerical stability from pivoting, the LU decomposition also has one other advantage: the elimination work is captured in $L$ and $U$ and does not have to be repeated if the same linear system must be solved repeatedly for different right-hand sides.

This is a good opportunity to use a *microbenchmark* to quickly get an idea whether this is true in practice. Let's create one using Jupyter's [`%%timeit`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) magic that measures the runtime of a single cell. First, set up a test problem, using again polynomial interpolation:

In [ ]:
n = 40
x = np.linspace( 0, 1, n )

# set up the system matrix
V = np.vander( x, increasing=True )

# create m random right-hand sides
m = 500
y = np.random.random(size=(m,n))

Now, we can measure the time required to do one decomposition and $m$ solutions:

In [ ]:
%%timeit

P,L,U = lu_factor(V)
for b in y:
    a = lu_solve(P,L,U, b)

<div class="alert alert-success">

**Task 4**: create a microbenchmark that uses `gauss_solve` to solve the system for each right-hand side.
</div>

In [ ]:
%%timeit

for b in y:
    gauss_solve(V, b) #given that gauss_solve takes in the raw matrix, V, we don't need to decompose it into P, L and U.
                      #we simply feed in the matrix, V, and use %%timeit to record the time it takes to solve the linear system for each 40 right-hand sides, b.
                      
                      #That takes considerably longer, given the expensive calculations that gauss_solve has to do (O(n^3) instead of O(n^2))

Depending on your implementation, the LU decomposition should be significantly faster on this particular problem.

Note: this is not a rigorous benchmark; it may be influenced by many factors such as overall system load etc. However, `%%timeit` can be useful to get a quick idea of execution time. (See the [documentation](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit) for more info.)

**Concluding remark**: While it was instructive for the purposes of this homework to implement Gauss elimination and LU decomposition by ourselves, for real-world use, NumPy's linear algebra module offers the routine [`numpy.linalg.solve`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html), which is a state-of-the-art linear system solver with many tricks to improve numerical stability. Furthermore, the `scipy` module contains implementations of the LU(P) decomposition (cf. [`scipy.linalg.lu`](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.linalg.lu.html)).

- - -

<div class="alert alert-info">
    
### Going to Space
</div>

So far, we have used polynomial interpolation only for one-dimensional function graphs of the form $y=p(x)$. However, the basic methodology carries easily to interpolating a curve in $n$ dimensions (e.g. in two dimensions).

We start from a modified basic setup: given $t_0,\dots,t_n$ and $x_0,\dots,x_n \in \mathbb{R}^d$, find the vector-valued polyomial $p(x)$ of degree $n+1$ such that 

$$p(t_i) = x_i.$$

It can be easily recognizes that this can be solved by finding $d$ separate one-dimensional (i.e. real-valued) polynomials $p_1,\dots,p_d$, each of degree $n+1$, for each of the $d$ components of the $x_i$. 

The following routine accomplishes this in the two-dimensional case, where `x` and `y` are arrays of coordinates of the interpolation points in the plane, and `t` is the array of the $t_i$ above.

In [ ]:
def poly2D(t, x, y):
    return np.linalg.solve( 
        np.vander( t, increasing=True ), 
        np.stack([x,y]).transpose() 
    )

Let's put this to the test:

In [ ]:
n = 20
t = np.linspace(0, 12, n)
x = (3+t) * np.cos( t )
y = (3+t) * np.sin( t )

a = poly2D( t, x, y )

# plot output
tt = np.linspace( t.min(), t.max(), 200 )
xx = np.polyval( np.flip(a[:,0]), tt )
yy = np.polyval( np.flip(a[:,1]), tt )

plt.plot( x, y, 'bo', xx, yy, 'r-' )
plt.axis("equal");

<div class="alert alert-success">

**Task 5**: Implement a function `poly3D`, by generalizing `poly2D` above to the three-dimensional case.
</div>

In [ ]:
# def poly3D(t, x, y, z):
#     # TODO
#     return np.empty_like(t)

def poly3D(t, x, y, z):
    return np.linalg.solve( 
        np.vander( t, increasing=True ),      # setting up the Vandermonde matrix
        np.stack([x,y,z]).transpose()         # stacking the arrays along the three axes
    )

<div class="alert alert-success">

**Task 6**: Illustrate the correctness of `poly3D` on a model problem similar to the one for `poly2D` above.

Notes: 
- To show the curve, you will have to use matplotlib's 3D plotting module. Documentation can be found [here](https://matplotlib.org/mpl_toolkits/mplot3d/tutorial.html), and a short tutorial is [here](https://towardsdatascience.com/an-easy-introduction-to-3d-plotting-with-matplotlib-801561999725).
- Using matplotlib's notebook mode (triggered by `%matplotlib notebook`) will give you a nice, interactively rotatable plot. Try it out!
</div>

In [ ]:
%matplotlib notebook     
# importing the notebook mode for the rotable plot

n = 20                     # number of points in the final plot
t = np.linspace(0, 12, n)
x = (3+t) * np.cos( t )    # setting up the formula for the points
y = (3+t) * np.sin( t )
z = (3+t) * np.sin( t )    # alternatively: z = (3+t) * np.cos( t ) for a different orientation in the 3d room

a = poly3D( t, x, y, z)    # using the poly3D function from task 5

# plot output
tt = np.linspace( t.min(), t.max(), 200 )   # setting up the lines, connecting the points
xx = np.polyval( np.flip(a[:,0]), tt )
yy = np.polyval( np.flip(a[:,1]), tt )
zz = np.polyval( np.flip(a[:,1]), tt ) # alternatively: zz = np.polyval( np.flip(a[:,1]), tt ), if one uses cos instead


ax = plt.axes(projection="3d")              # initializing the 3d coordinate system
ax.scatter3D(x, y, z, c=z, cmap='hsv')      # adding the points
ax.plot3D(xx, yy, zz)                       # adding the line
plt.show()